# OAuth Authorization Code Flow — Sequence Reasoning

This notebook explains OAuth 2.0 Authorization Code flow
using text-based sequence diagrams.

The goal is to understand:
- who talks to whom
- who trusts whom
- where each security invariant is enforced

No code is used in this notebook.


## Actors in the OAuth System

We always reason with *roles*, not implementations.

1. Client (Agent)
   - Wants to access a protected service
   - Identity is its client_id (URL in CIMD)

2. Authorization Server (AS)
   - Central authority
   - Authenticates clients and users
   - Issues tokens

3. Resource Server (Service / Tool)
   - Hosts protected data or actions
   - Trusts tokens issued by AS


## High-Level OAuth Flow

Client → Authorization Server → Client → Resource Server


## Authorization Code Flow (Sequence Diagram)

(1) Client → Authorization Server
    Request authorization
    Includes:
      - client_id
      - redirect_uri
      - scope
      - PKCE code_challenge

(2) Authorization Server → Client (User interaction)
    Prompts user or policy decision

(3) Authorization Server → Client (redirect)
    Redirects back with:
      - authorization code

(4) Client → Authorization Server
    Exchanges authorization code
    Includes:
      - code
      - PKCE code_verifier

(5) Authorization Server → Client
    Issues access token

(6) Client → Resource Server
    Calls API with:
      Authorization: Bearer <access_token>

(7) Resource Server
    Verifies token and scope
    Returns protected data


## Where Each Invariant Is Enforced

Invariant 1 (HTTPS client_id)
→ Step (1): AS validates client_id scheme

Invariant 2 (Metadata fetched from client_id URL)
→ Step (1): AS fetches metadata (CIMD)

Invariant 3 (Metadata self-identifies client_id)
→ Step (1): AS validates metadata

Invariant 4 (Exact redirect URI match)
→ Step (1): AS validates redirect_uri

Invariant 5 (Metadata caching & revocation)
→ Step (1): AS cache controls

Invariant 6 (PKCE mandatory)
→ Steps (1) and (4)

Invariant 7 (Server is final authority)
→ Steps (2) and (5)


## What the Resource Server Must NEVER Do

The Resource Server:

- ❌ Does NOT fetch client metadata
- ❌ Does NOT know client_id
- ❌ Does NOT authenticate the client
- ❌ Does NOT handle PKCE

It ONLY:
- verifies token signature
- verifies token issuer
- verifies scopes


## What Breaks Without an Authorization Server

If the client talks directly to the service:

- No central policy enforcement
- No consistent identity
- No revocation
- No audit trail
- No separation of concerns

OAuth exists to prevent this.


## One-Page OAuth Flow Summary

- Client never authenticates directly to Service
- Authorization Server is the trust anchor
- Tokens are the delegation mechanism
- Services trust tokens, not identities
- CIMD only affects client → AS interaction
